In [1]:
def inverse_standardize(data, means, stds):
    """
    对标准化数据进行反标准化。

    参数：
    data: np.ndarray - 标准化后的数据。
    means: np.ndarray - 每个通道的均值。
    stds: np.ndarray - 每个通道的标准差。

    返回：
    original_data: np.ndarray - 反标准化后的数据。
    """
    original_data = data * stds + means
    return original_data


In [8]:
import numpy as np
mean = np.load('../data/dt1/mean.npy')
std = np.load('../data/dt1/std.npy')
pre_data_1=np.load('data/MultiOutput/predictions.npy')
# 对每个通道（mode）进行反标准化
# mean[i] 和 std[i] 分别为每个 mode 的均值和标准差
predictions_inverse = np.zeros_like(pre_data_1)  # 创建一个与 pre_data_1 形状相同的数组
for i in range(pre_data_1.shape[1]):
    predictions_inverse[:, i] = inverse_standardize(pre_data_1[:, i], mean[i], std[i])

# 保存反标准化后的数据
np.save('data/MultiOutput/predictions_inverse.npy', predictions_inverse)
print("反标准化后的数据已保存为 'data/MultiOutput/predictions_inverse.npy'")

反标准化后的数据已保存为 'data/MultiOutput/predictions_inverse.npy'


In [9]:
import numpy as np
from Ablation.model import AnticipationModule
def anticipate_mode_ori(data_path):
    predication = np.load(data_path)
    anticipation_module = AnticipationModule()
    prediction_data = anticipation_module(predication)
    return prediction_data
anticipate_data = anticipate_mode_ori('data/MultiOutput/predictions_inverse.npy')
print(anticipate_data.shape)
np.save('data/MultiOutput/predictions_inverse_final', anticipate_data)

(600, 1)


In [10]:
import pandas as pd
import numpy as np
def extract_label():
    file_path = 'E:/Dataset/waves/'
    file_path_ori = file_path + 'dataset1.csv'
    data = pd.read_csv(file_path_ori)
    # 忽略 'Date/Time' 列
    if 'Date/Time' in data.columns:
        data = data.drop(columns=['Date/Time'])
    mwh_data = data['MWH'].values[-600:]  # 提取最后 600 行的 MWH 列
    mwh_data=mwh_data.reshape(-1, 1) 
    return mwh_data
original_data = extract_label()
original_data.shape

(600, 1)

In [11]:
from Ablation.model import ModelEvaluator

#源数据的数据评测
predictions_inverse_final=np.load('data/MultiOutput/predictions_inverse_final.npy')
evaluator = ModelEvaluator(anticipate_data, original_data)
print(f"Root Mean Squared Error (RMSE): {evaluator.rmse()}")
print(f"Mean Absolute Error (MAE): {evaluator.mae()}")
print(f"Sum of Squared Errors (SSE): {evaluator.sse()}")
print(f"Mean Absolute Percentage Error (MAPE): {evaluator.mape()}")
print(f"Theil's Inequality Coefficient (TIC): {evaluator.tic()}")

Root Mean Squared Error (RMSE): 0.2909750699869967
Mean Absolute Error (MAE): 0.1910496585528056
Sum of Squared Errors (SSE): 50.79989481236257
Mean Absolute Percentage Error (MAPE): 7.271417070537563
Theil's Inequality Coefficient (TIC): 0.11696020916752838
